In [1]:
import pandas as pd
import json
import numpy as np
from os import listdir
from io import StringIO
from Bio import Phylo
import re

In [6]:
path="/home/maeperez/scratch/Clams/hyphy2.5/gard"
path_out="/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data/"

genes=[]
for file in listdir(path):
    if file[-4:]=="json":
        genes+=[str(file.split('.')[0])]
genes=sorted(genes)
print(len(genes))

638


In [9]:
with_recombination=[]
for gene in genes:
    remote_file = open(path+'/'+gene+'.GARD.json')
    gard_json = json.load(remote_file)
    
    if 'improvements' not in gard_json.keys():
        ### write gard_splits output for KH test (has to be done with the older version of hyphy? )
        entry='\n\n'
        entry+='-'.join(map(str,([i-1 for i in gard_json['breakpointData']['bps'][0]])))+'\n'+gard_json['breakpointData']['tree']+'\n'
        with open(path+'/'+gene+'.gard_splits','w') as fa:
            fa.write(entry) 
        continue
        
    else:
#         max_bps=[gard_json['improvements'][bp]['breakpoints'] for bp in list(gard_json['improvements'].keys())[-1]][0]
#         max_part=int(list(gard_json['improvements'].keys())[-1])+1

        ### write gard_splits output for KH test (has to be done with the older version of hyphy? )
        entry='\n\n'
        for n in gard_json['breakpointData'].keys():
            entry+='-'.join(map(str,([i-1 for i in gard_json['breakpointData'][str(n)]['bps'][0]])))+'\n'+gard_json['breakpointData'][str(n)]['tree']+'\n'
        with open(path+'/'+gene+'.gard_splits','w') as fa:
            fa.write(entry)   

        best_AICc=sum([gard_json['improvements'][bp]['deltaAICc'] for bp in gard_json['improvements'].keys()])+gard_json['baselineScore']
        AICc_ratio=float(gard_json['singleTreeAICc'])/best_AICc
        if AICc_ratio>=1:
            with_recombination+=[gene]
print( 'number of genes that failed AICc test ',len(with_recombination),"/",len(genes))        

number of genes that failed AICc test  143 / 638


In [10]:
%%bash
# Run KH test ...

sbatch run_array_KHtest.sl

Submitted batch job 15610872


In [13]:
#### PARSE KH test results ####
## GROUP1 ##
path="/home/maeperez/scratch/Clams/hyphy2.5/gard/"
onlyfiles = [path+f for f in listdir(path) if f[-7:]=='.KHtest']
kh=pd.DataFrame([],columns=['Gene','p0.01','p0.05','p0.1'])
print( 'number of genes that ran the KH test',len(onlyfiles))


number of genes that ran the KH test 637


In [12]:
set([re.split('/|\.|',file)[-2] for file in onlyfiles])-set(genes)

set()

In [13]:
for i in range(len(onlyfiles)):
# for i in [0]:
    with open(onlyfiles[i]) as f:
        gene=re.split('/|\.|',onlyfiles[i])[-2]
        lines=f.read().splitlines()
        for n in reversed(range(len(lines))):
            if lines[n][:6]=='At p =':
#                 print lines[n-2]
#                 print lines[n-1]
#                 print lines[n]
                try:
                    p001= int(lines[n-2].split(' ')[-3])
                    p005= int(lines[n-1].split(' ')[-3])
                    p01= int(lines[n].split(' ')[-3])
                except:
                    p001= np.nan
                    p005= np.nan
                    p01= np.nan
                break
        newentry=dict(zip(['Gene','p0.01','p0.05','p0.1'],[gene,p001,p005,p01]))
        kh=kh.append(newentry,ignore_index=True)


print( 'number of genes that failed KH test ',len(kh.loc[kh['p0.05']>0]))
print( 'number of genes that failed AICc test ',len(with_recombination))

number of genes that failed KH test  152
number of genes that failed AICc test  143


In [14]:
kh['AICctest']=0
kh.loc[kh['Gene'].isin(with_recombination),'AICctest']=1
kh.to_csv(path_out+'gard_results.txt')


NameError: name 'sq' is not defined

In [18]:
recomb_KH=kh.loc[kh['p0.05']>0]['Gene']
print('recomb with AICc only',len(set(with_recombination)-set(recomb_KH)))
print('recomb with KH only',len(set(recomb_KH)-set(with_recombination)))
print('recomb with KH and AICc',len(set(recomb_KH).intersection(set(with_recombination))))



recomb with AICc only 116
recomb with KH only 125
recomb with KH and AICc 27


In [39]:
print('%s/%d genes show evidence for recombination' % (len(with_recombination),len(genes)))
# 85/389 genes show evidence for recombination: hyphy2.5

0/508 genes show evidence for recombination


In [50]:
df=pd.read_csv('/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data_ms1/KHtest_results.txt',sep=',',header=0,index_col=0)
df[df['Gene'].isin(with_recombination) & df['p0.05']>0]

print('genes in common between older version and hyphy2.5')
print(len(set(genes).intersection(set(df['Gene'].values))))
print('genes with signficant recomb in hyphy2.5 but not older version')
print(len(set(with_recombination)-set(df[df['p0.05']>0]['Gene'].values)))
print('genes with significant recomb older version but not hyphy 2.5')
print(len(set(df[df['Gene'].isin(genes) & df['p0.05']>0]['Gene'].values)-set(with_recombination)))
print('genes with significant recomb in both version')
print(len(set(with_recombination).intersection(set(df[df['p0.05']>0]['Gene'].values))))


genes in common between older version and hyphy2.5
351
genes with signficant recomb in hyphy2.5 but not older version
81
genes with significant recomb older version but not hyphy 2.5
30
genes with significant recomb in both version
4


In [62]:
print(len('.GTR.gard.json'))

14


In [56]:
print(path+'/'+gene+'_'+str(n)+'.tree')

/home/maeperez/scratch/Clams/hyphy2.5/gard/Rmag_0003_1.tree
